# dummy Parsing


In [48]:
import re
import unicodedata
import os

In [2]:
# Pfade zu Quelldatei und Zieldatei

src_path = r"C:\Users\daniil\Desktop\proj_landeskunde\data\hol\txt\HOL_BB_6.txt"
dst_path = r"C:\Users\daniil\Desktop\proj_landeskunde\data\hol\txt\HOL_BB_6_normalized.txt"

# Datei laden
with open(src_path, "r", encoding="utf-8") as f:
    raw = f.read()

# Unicode normalisieren (z.B. Trennstriche, Akzente konsistent machen)
clean = unicodedata.normalize("NFKC", raw)

# Seitenangaben entfernen ("=== Seite 1 ===")
clean = re.sub(r"===\s*Seite\s*\d+\s*===", "", clean)

# Alle mehrfachen Leerzeilen auf max. 1 reduzieren
clean = re.sub(r"\n{2,}", "\n\n", clean)

# Whitespace trimmen
clean = clean.strip()

# Normalisierte Datei speichern
with open(dst_path, "w", encoding="utf-8") as f:
    f.write(clean)

print("Normalisierte Datei gespeichert unter:")
print(dst_path)


Normalisierte Datei gespeichert unter:
C:\Users\daniil\Desktop\proj_landeskunde\data\hol\txt\HOL_BB_6_normalized.txt


In [6]:
# Blöcke schneiden
blocks = {}

for idx, (place, start) in enumerate(places):
    end = places[idx + 1][1] if idx + 1 < len(places) else len(lines)
    block = "\n".join(lines[start:end]).strip()
    blocks[place] = block



In [7]:
blocks["AHRENSFELDE"]
# -> kompletter Textblock von AHRENSFELDE

'AHRENSFELDE \ns Bernau \nNiederbarnim - Niederbarnim - Kr Bernau/Bez Frankfurt. \n1. Dorf, GemBez mit Ausbau Neu Ahrensfelde, 1927 mit Bahnhof \nund Häusergruppe Berliner Ostfriedhof; 1932 LandGem mit 5 \nWohnplätzen: Siedlungen Block A, Block B, Block C und \nBlock D, Bahnhof Ahrensfelde Friedhof; 1950, 1957 mit \nWohnplatz Ausbau 1973 Gem. \n2. 1660: 3762 Mg (9B Mg Gehöfte, 3091 Mg Acker, 329 Mg Wiese, \n244 Mg Wald), 1900: 977 ha, 1931: 759 ha. \n3. Straßendorf {UrMBl 1838/3447 Friedrichsfelde von etwa \n1840); jüngerer Kietz im 0 des Ortes an der Straße nach \nMehrow (Krüger s. 139). \n4. 1375 Arnsfelde, Arnsfelt (Landbuch s. 69, 116). \n5. Bis 1ö49 LandG Berlin, 1849-1878 KreisG Berlin, 1879-1906 \nAmtsG Berlin II, 1906-1945 AmtsG Weißensee bzw., seit 1912, \nBerlin-Weißensee, 1945-1952 AmtsG Bernau. \n6. Um 1375 die v.Oderberg über OberG, Wagendienst und 29 Hf \nzu ihren Höfen (daneben Bürger Lietzen in Berlin über \nPacht, Zins und Bede von 8 Hf, noch 1441 über Hebungen in \nA.

## 1. Parser "places — Zentrale Ortstabelle"

In [5]:
import re

lines = clean.split("\n")
places = []

# Erst: alle Ortsüberschriften und deren Zeilennummer sammeln
for i, line in enumerate(lines):
    line = line.strip()
    if re.fullmatch(r"[A-ZÄÖÜ0-9' ()\/\-]+", line):
        if len(line) > 2 and not line.lower().startswith("s."):
            places.append((line, i))



# erste 20 anzeigen
for p in places[:20]:
    print(p)

("KRAUSE'S ABBAU (WILHELMSHOF)", 0)
('ABDECKEREI', 2)
('KOCHS ABLAGE', 4)
('ACKERMANNSHOF', 6)
('ADDERLUCH', 33)
('AHRENSFELDE', 36)
('NEU AHRENSFELDE', 147)
('ALBERTSHOF', 242)
('ALBRECHTSGELÄNDE', 267)
('KOLONIE ALPENBERGE', 272)
('ALTENA', 274)
('ALTENHOF', 333)
('ALTENHOF', 335)
('ALTENHOF', 337)
('AL YENSLEBENBRUCH', 339)
('AMALIENHOF', 344)
('AMALIENHOF', 381)
('AMALIENHOF', 383)
('AMALIENHOF', 385)
('LANDSEERGER AMTS-MEIEREI', 388)


In [34]:
#type parser

def extract_type(block: str):
    m = re.search(r"\b1\.\s*([A-Za-zÄÖÜäöüß]+)", block)
    if m:
        return m.group(1)
    return None

print(extract_type(blocks["AHRENSFELDE"]))

Dorf


In [35]:
# type settlement parser

def extract_settlement_type(block: str):
    m = re.search(r"\b3\.\s*([A-Za-zÄÖÜäöüß]+)", block)
    if m:
        return m.group(1)
    return None

print(extract_settlement_type(blocks["AHRENSFELDE"]))


Straßendorf


In [36]:
# first mention year parser

def extract_first_mention_year(block: str):
    m = re.search(r"\b4\.\s*(\d{3,4})", block)
    if m:
        return int(m.group(1))
    return None

print(extract_first_mention_year(blocks["AHRENSFELDE"]))


1375


In [37]:
# first mention source parser

def extract_first_mention_source(block: str):
    # Klammerinhalt hinter Punkt 4
    m = re.search(r"\b4\.[^\(]*\(([^)]+)\)", block)
    if m:
        return m.group(1).strip()
    return None

print(extract_first_mention_source(blocks["AHRENSFELDE"]))



Landbuch s. 69, 116


In [38]:
# lage_hinweis parser

def extract_lage_hinweis(block: str):
    lines = block.strip().splitlines()
    if len(lines) >= 2:
        return lines[1].strip()
    return None

print(extract_lage_hinweis(blocks["AHRENSFELDE"]))



s Bernau


In [39]:
# Kompletter Parser für einen Ortsblock

def parse_place(block: str):
    return {
        "type": extract_type(block),
        "settlement_type": extract_settlement_type(block),
        "first_mention_year": extract_first_mention_year(block),
        "first_mention_source": extract_first_mention_source(block),
        "lage_hinweis": extract_lage_hinweis(block)
    }


In [40]:
result = parse_place(blocks["AHRENSFELDE"])
print(result)


{'type': 'Dorf', 'settlement_type': 'Straßendorf', 'first_mention_year': 1375, 'first_mention_source': 'Landbuch s. 69, 116', 'lage_hinweis': 's Bernau'}


In [41]:
# Case-insensitive dict für Ortsdaten

class PlaceDict(dict):
    def __init__(self, *args, **kwargs):
        super().__init__()
        for k, v in dict(*args, **kwargs).items():
            self[k.upper()] = v

    def __dir__(self):
        return list(self.keys())

    def __getattr__(self, item):
        item = item.upper()
        if item in self:
            return self[item]
        raise AttributeError(item)

    def __getitem__(self, item):
        return super().__getitem__(item.upper())



In [42]:
# Beispielnutzung, automatisch ausfüllbar durch Tab-Vervollständigung

blocks["BERNAU"]  

'BERNAU \nNiederbarnim - Niederbarnim - Kr Bernau/Bez Frankfurt. \n1. Stadt, StadtGem mit Kol Birkbuscb, Vw Karlslust, Wasser-\nmühle Malzmühle und Försterei Schmetzdorf, 1927 mit An-\nsiedlung Neu Bernau, Ansiedlung Friedensthai und Wohnbaus \nWaldkater; 1928 Teil des GutsBez Schmatzdorf eingemeindet; \n1932 Stadt mit den Wohnplätzen Anglersruh, Birkbusch, Bun-\ndesschule, Gieses Plan, Karlslust, Kleine Mühle, Gasthaus \nLiepnitz, Neu Bernau, Sägewerk, Waldkater, den Forsthäu-\nsern Liepnitz und Schmatzdorf und den Kol Bernau-Friedens-\nthal, Bernau Nord, Bernau Süd, Birkbolztal und Wudo; \n1950 Stadt mit den Wohnplätzen Anglersruh, Birkenhöhe, \nBlumenbag1 Eichwerder, Friedensthal, Liepnitz Försterei, \nLindow, N~belungen, Pankeborn, Rehberge einscbl. Karls-\nlust, Rutenfeld und Waldfrieden; \n1957, 1973 Kreisstadt mit den Ortsteilen Birkenböbe, Blu-\nmenhag, Eicbwerder, Friedenstbal, Nibelungen, Pankeborn \n31 \np\n\nund Waldfrieden, 1973 auch mit den Ortsteilen Lindow und \nRutenfe

## 2. Parser "place_aliases — Historische Namensformen"

In [43]:
# Alias parser

def parse_aliases(block_text):
    p4 = re.search(r"4\.\s*(.+?)(?=\n\d+\.)", block_text, flags=re.S)
    if not p4:
        return []

    text = p4.group(1).strip()

    parts = [p.strip() for p in re.split(r"\)\s*,", text)]
    aliases = []

    for part in parts:
        year_match = re.match(r"(\d{3,4})\s+(.*)", part)
        if not year_match:
            continue

        year = int(year_match.group(1))
        rest = year_match.group(2)

        name_match = re.match(r"([^()]+?)\s*\((.*)\)", rest)
        if not name_match:
            continue

        alias_names_raw = name_match.group(1).strip()
        source_raw = name_match.group(2).strip()

        alias_list = [a.strip() for a in alias_names_raw.split(",")]

        for alias in alias_list:
            aliases.append({
                "alias": alias,
                "year_from": year,
                "year_to": None,
                "source": source_raw
            })

    return aliases


In [44]:
# Beispielnutzung des Alias-Parsers

def print_aliases(place_name):
    if place_name not in blocks:
        print(f"Ort '{place_name}' nicht gefunden!")
        return

    block_text = blocks[place_name]
    aliases = parse_aliases(block_text)

    if not aliases:
        print(f"Keine Alias-Namen für {place_name} gefunden.")
        return

    print(f"Alias-Namen für {place_name}:")
    for a in aliases:
        print(f"- {a['alias']} ({a['year_from']}), Quelle: {a['source']}")


In [45]:
# Beispielaufruf

print_aliases("AHRENSFELDE")


Alias-Namen für AHRENSFELDE:
- Arnsfelde (1375), Quelle: Landbuch s. 69, 116
- Arnsfelt (1375), Quelle: Landbuch s. 69, 116


## 3. Parser place_admin_history — Verwaltungsgeschichte

In [46]:
# Verwaltungszuordnungs-Parser

def parse_admin(block_text):
    lines = block_text.split("\n")

    # Verwaltungseintrag finden (erste Zeile mit "-")
    admin_line = None
    for line in lines:
        if " - " in line:
            admin_line = line.strip().rstrip(".")
            break

    if not admin_line:
        return []

    # Teile an " - "
    parts = [p.strip() for p in admin_line.split(" - ")]

    results = []

    # --- Phase 1: vor 1816 ---
    results.append({
        "year_from": None,
        "year_to": 1815,
        "admin_unit": parts[0],
        "admin_level": "Kreis",
        "notes": ""
    })

    # --- Phase 2: 1816–1952 ---
    results.append({
        "year_from": 1816,
        "year_to": 1951,
        "admin_unit": parts[1],
        "admin_level": "Kreis",
        "notes": ""
    })

    # --- Phase 3: ab 1952 ---
    last = parts[2]

    # z.B. "Kr Bernau/Bez Frankfurt"
    sub = [p.strip() for p in re.split(r"[\/]", last)]

    for s in sub:
        if s.startswith("Kr "):
            results.append({
                "year_from": 1952,
                "year_to": 1990,
                "admin_unit": s[3:].strip(),
                "admin_level": "Kreis",
                "notes": ""
            })
        elif s.startswith("Bez "):
            results.append({
                "year_from": 1952,
                "year_to": 1990,
                "admin_unit": s[4:].strip(),
                "admin_level": "Bezirk",
                "notes": ""
            })
        else:
            # fallback
            results.append({
                "year_from": 1952,
                "year_to": 1990,
                "admin_unit": s,
                "admin_level": "Unbekannt",
                "notes": ""
            })

    return results


In [47]:
# Beispielaufruf

def print_admin(place_name):
    if place_name not in blocks:
        print(f"Ort '{place_name}' nicht gefunden!")
        return

    admin_data = parse_admin(blocks[place_name])

    print(f"Verwaltungs-Historie für {place_name}:\n")
    for row in admin_data:
        print(f"- {row['year_from']}–{row['year_to']}: "
              f"{row['admin_unit']} ({row['admin_level']})")


In [48]:
# Beispielaufruf

print_admin("AHRENSFELDE")


Verwaltungs-Historie für AHRENSFELDE:

- None–1815: Niederbarnim (Kreis)
- 1816–1951: Niederbarnim (Kreis)
- 1952–1990: Bernau (Kreis)
- 1952–1990: Frankfurt (Bezirk)


## 4. Parser place_court_history

In [49]:
# Gerichtshistorie-Parser

def parse_court_history(text):
    """
    Erwartet den gesamten Block des Ortes (blocks['AHRENSFELDE'])
    Extrahiert daraus automatisch Punkt 5.
    """
    # 1) Finde Punkt 5
    m = re.search(r"5\.\s*(.*?)\n(?=\d+\.|$)", text, flags=re.S)
    if not m:
        return []

    raw = m.group(1).replace("\n", " ")

    # 2) Split an Kommas/Semikolons
    parts = re.split(r"[;,]\s*", raw)

    entries = []
    for part in parts:
        part = part.strip()
        if not part:
            continue

        # "Bis 1849"
        m1 = re.match(r"Bis\s+(\d{4})\s+(.*)", part)
        if m1:
            year_to = int(m1.group(1))
            rest = m1.group(2)
            tokens = rest.split(maxsplit=1)
            court_type = tokens[0]
            court_name = rest
            entries.append({
                "year_from": None,
                "year_to": year_to,
                "court_type": court_type,
                "court_name": court_name
            })
            continue

        # "1849–1878"
        m2 = re.match(r"(\d{4})\s*[-–]\s*(\d{4})\s+(.*)", part)
        if m2:
            yf = int(m2.group(1))
            yt = int(m2.group(2))
            rest = m2.group(3)
            tokens = rest.split(maxsplit=1)
            court_type = tokens[0]
            court_name = rest
            entries.append({
                "year_from": yf,
                "year_to": yt,
                "court_type": court_type,
                "court_name": court_name
            })
            continue

    return entries


In [50]:
# Beispielaufruf

def print_court_history(place_name):
    if place_name not in blocks:
        print(f"Ort '{place_name}' nicht gefunden!")
        return

    court_data = parse_court_history(blocks[place_name])

    print(f"Gerichtszugehörigkeit für {place_name}:\n")
    for row in court_data:
        yf = row["year_from"] if row["year_from"] is not None else "?"
        yt = row["year_to"] if row["year_to"] is not None else "?"

        print(f"- {yf}–{yt}: {row['court_name']} "
              f"({row['court_type']})")


In [51]:
print_court_history("AHRENSFELDE")

Gerichtszugehörigkeit für AHRENSFELDE:

- 1849–1878: KreisG Berlin (KreisG)
- 1879–1906: AmtsG Berlin II (AmtsG)
- 1906–1945: AmtsG Weißensee bzw. (AmtsG)
- 1945–1952: AmtsG Bernau. (AmtsG)


In [52]:
print_court_history("BERNAU")

Gerichtszugehörigkeit für BERNAU:

- 1839–1849: Land- und StadtG B. (durch  Vereinigung mit Justizamt Biesenthal) (Land-)
- 1849–1878: GKomm B. (GKomm)


## 5. place_area — Fläche / Gemarkung

In [53]:
# Flächenangaben-Parser (generisch)

def parse_area_generic(block_text):
    """
    Extrahiert alle Flächenangaben aus Punkt 2 eines HOL-Eintrags.
    Generisch: alle Kategorien in den Klammern werden übernommen.
    Gibt Liste von Datensätzen zurück.
    """

    # Punkt 2 isolieren
    m = re.search(r"\b2\.\s+(.*?)(?=\n\s*\d+\.)", block_text, re.S)
    if not m:
        return []

    text = m.group(1)
    entries = []

    # Jahr + Gesamt + optional Kategorien
    pattern = r"(\d{3,4})\s*:\s*([\d\s]+)\s*(Mg|ha)(?:\s*\((.*?)\))?"
    for year, total, unit, details in re.findall(pattern, text, re.S):
        total = int(total.replace(" ", ""))

        row = {
            "year": int(year),
            "area_total": total,
            "unit": unit,
        }

        # Falls Kategorien angegeben sind (Klammerinhalt)
        if details:
            # Alle Teile trennen: "360 Mg Gehöfte, 72 Mg Gartenland, 8758 Mg Acker ..."
            parts = re.findall(r"(\d[\d\s]*)\s*Mg\s+([A-Za-zäöüÄÖÜß]+)", details)
            for value, name in parts:
                clean_value = int(value.replace(" ", ""))
                # Name direkt übernehmen
                row[name] = clean_value

        entries.append(row)

    return entries

# Beispielaufruf
def print_area(place_name):
    if place_name not in blocks:
        print(f"Ort '{place_name}' nicht gefunden!")
        return

    area_data = parse_area_generic(blocks[place_name])
    print(f"Flächenangaben für {place_name}:\n")
    for row in area_data:
        print(f"- {row['year']}: {row['area_total']} {row['unit']}")
        for k, v in row.items():
            if k in ("year", "area_total", "unit"):
                continue
            print(f"    • {k}: {v} {row['unit']}")


In [54]:
# Beispielaufruf

def print_area(place_name):
    if place_name not in blocks:
        print(f"Ort '{place_name}' nicht gefunden!")
        return

    area_data = parse_area_generic(blocks[place_name])

    print(f"Flächenangaben für {place_name}:\n")
    for row in area_data:
        print(f"- {row['year']}: {row['area_total']} {row['unit']}")

        # Kategorien drucken
        for k, v in row.items():
            if k in ("year", "area_total", "unit"):
                continue
            print(f"    • {k}: {v} {row['unit']}")



In [55]:
# Beispielaufruf

print_area("AHRENSFELDE")

Flächenangaben für AHRENSFELDE:

- 1660: 3762 Mg
    • Acker: 3091 Mg
    • Wiese: 329 Mg
    • Wald: 244 Mg
- 1900: 977 ha
- 1931: 759 ha


In [56]:
print_area("BERNAU")

Flächenangaben für BERNAU:

- 1860: 19622 Mg
    • Gehöfte: 360 Mg
    • Gartenland: 72 Mg
    • Acker: 8758 Mg
    • Wiese: 168 Mg
    • Weide: 1246 Mg
    • Torf: 18 Mg
    • Wald: 9000 Mg
- 1900: 4643 ha
- 1931: 4703 ha


## 6. Parsing Verwaltung

In [57]:
import re

def parse_lordship(block_text):
    """
    Extrahiert Herrschafts-/Lehnsangaben (Punkt 6) aus einem HOL-Eintrag.
    Gibt Liste von dicts zurück:
    {
        year_from,
        year_to,
        owner,
        share_type
    }
    """

    # Punkt 6 isolieren
    m = re.search(r"\b6\.\s+(.*?)(?=\n\s*\d+\.)", block_text, re.S)
    if not m:
        return []

    text = m.group(1).strip()

    # Zerlegen in Segmente, typischerweise durch ';' getrennt
    segments = re.split(r";\s*", text)

    results = []

    for seg in segments:
        seg = seg.strip()
        if not seg:
            continue

        # --- Jahresangaben extrahieren ---
        # Beispiele:
        #   1375
        #   1490/1499–1672
        #   vor 1450 bis nach 1490
        #   1717-1728
        #   um 1375
        year_pattern = r"(um\s*\d{3,4}|vor\s*\d{3,4}|nach\s*\d{3,4}|\d{3,4}(?:/\d{3,4})?(?:[-–]\d{3,4})?)"
        years = re.findall(year_pattern, seg)

        year_from = None
        year_to = None

        if years:
            yr = years[0]

            # Fälle behandeln
            if "um" in yr:
                y = int(re.search(r"\d+", yr).group())
                year_from = y - 5
                year_to = y + 5

            elif "vor" in yr:
                y = int(re.search(r"\d+", yr).group())
                year_from = None
                year_to = y

            elif "nach" in yr:
                y = int(re.search(r"\d+", yr).group())
                year_from = y
                year_to = None

            elif "/" in yr:
                # 1490/1499–1672
                left, rest = yr.split("/")
                left2 = re.split(r"[-–]", rest)
                year_from = int(left2[0])
                year_to = int(left2[1]) if len(left2) > 1 else None

            elif "-" in yr or "–" in yr:
                y1, y2 = re.split(r"[-–]", yr)
                year_from = int(y1)
                year_to = int(y2)

            else:
                # Einzeljahr
                year_from = int(yr)
                year_to = None

        # --- Besitzer extrahieren ---
        # Muster: beginnt meist nach der Jahresangabe
        # Beispiele:
        #   "die v.Oderberg"
        #   "Bürger Stroband in Berlin"
        #   "Amt Mühlenhof"
        #   "Kloster Zehdenick"
        owner_pattern = r"(v\.[A-Za-z]+|Frh\.v\.[A-Za-z]+|Amt [A-Za-zÄÖÜäöüß]+|Kloster [A-Za-zÄÖÜäöüß]+|Bürger [A-Za-zÄÖÜäöüß]+(?: in [A-Za-zÄÖÜäöüß]+)?)"
        owner_m = re.search(owner_pattern, seg)

        owner = owner_m.group(1) if owner_m else None

        # --- Rest als share_type ---
        share_text = seg
        if years:
            share_text = share_text.replace(years[0], "")
        if owner:
            share_text = share_text.replace(owner, "")

        share_text = share_text.strip(" ,.-")

        results.append({
            "year_from": year_from,
            "year_to": year_to,
            "owner": owner,
            "share_type": share_text
        })

    return results


In [58]:
def print_lordship(place_name):
    if place_name not in blocks:
        print("Ort nicht gefunden.")
        return

    data = parse_lordship(blocks[place_name])

    print(f"Herrschaftsverhältnisse für {place_name}:\n")
    for r in data:
        print(f"- {r['year_from']} – {r['year_to']} | {r['owner']} | {r['share_type']}")


In [59]:
print_lordship("AHRENSFELDE")

Herrschaftsverhältnisse für AHRENSFELDE:

- 1375 – None | v.Oderberg | Um  die  über OberG, Wagendienst und 29 Hf 
zu ihren Höfen (daneben Bürger Lietzen in Berlin über 
Pacht, Zins und Bede von 8 Hf, noch 1441 über Hebungen in 
A
- None – None | Bürger Krüger in Bernau | die Queste,  über Hebungen
- 1590 – None | Kloster Zehdenick | dgl 

 über Hebungen, noch  Amt Zehdenick 
über Hebungen von 2 Leuten
- None – None | v.Sch | Altar in Biesenthal über Pacht 
und Zins von 4 Hf, Witwe Schlaberndorf über 3 Hf auf Le-
benszeit, fällt danach an H.önfeld)
- None – 1450 | Bürger Stroband in Berlin | bis nach 1490  über das 
Dorf mit Ober- und UnterG (1451) und Patr (1472, 1490)
- 1499 – 1672 | Amt Mühlenhof | 
  über das Dorf mit Ober- und 
UnterG und Patr (1541, 1591), im 17. Jh nur über das halbe 
Dorf (1608) bzw. über das Dorf ohne die Dienste, daneben 
Vorwerk Marzahn über das halbe Dorf (1608) bzw. über die 
Dienste (161c.., 1700)
- 1721 – None | Amt Mühlenhof | im 1i:..,. und 19. Jh  wie

In [60]:
print_lordship("BERNAU")

Herrschaftsverhältnisse für BERNAU:

- 1375 – None | None | Landesherrliche Immediatstadt ( Markgraf über OberG, 
Patr, Urbede und andere Abgaben)
- 1315 – None | None | Consules, seit 15. 
Jb Bürgermeister, seit Polizeiordnung von 1515: 16 Rats-
mitglieder (2 Bürgermeister 1 Richter, 3 Ratmänner, 2 
Kämmerer jährlich wechselnd~
- 1706 – None | None | · zwei Bürgermeister, 1 
Richter und zugleich Kämmerer, 6 Ratmänner
- 1719 – None | None | seit  stän-
diger Magistrat: 1 Stadtdirektor, 1 Polizeibürgermeister, 
22 Ratmänner, der französische KolRichter, bis 1795 noch 
ein Kämmerer
- 1249 – None | None | seit 1c09: 2 besoldete, 4 (seit : 6) un-
besoldete Mitglieder
- 1795 – 1805 | None | Viergewerke und ganze Gemeinde als 
Vertreter der Bürgerschaft, im 16. Jb 20, später .c: Verord-
nete (4 Viertelsmeister),  ganz vom Magistrat ab-
hängig


## 7. parse_economy

In [9]:
# Wirtschaftparser

def parse_economy(block_text):
    """
    Extrahiert Wirtschaftsangaben aus Punkt 7/8 eines HOL-Eintrags.
    Gibt Liste von Economy-Objekten im JSON-Format zurück.
    """

    # ---- Punkt 7 isolieren ----
    m = re.search(r"\b7\.\s+(.*?)(?=\n\s*\d+\.)", block_text, re.S)
    if not m:
        return []
    text = m.group(1).strip()

    # ---- Jahresblöcke trennen ----
    parts = re.split(r"(?:(?<=\n)|^)\s*(\d{3,4}[a-z]?)\s*:\s*", text)

    results = []
    last_entry = None

    # parts = ["", year1, text1, year2, text2, ...]
    for i in range(1, len(parts), 2):
        year_raw = parts[i]
        content = parts[i+1].strip()

        # OCR-Korrekturen
        year_clean = year_raw.replace("c", "0").replace("·", "")

        if not year_clean.isdigit():
            continue

        year = int(year_clean)

        entry = {"year": year}

        # ---- 1. Spezialfall: dgl. = wie vorher ----
        if "dgl" in content.lower() and last_entry:
            entry.update({k: v for k, v in last_entry.items() if k != "year"})

        # ---- 2. Gesamthufen extrahieren ----
        m_hf_total = re.search(r"(\d+)\s*Hf\b", content)
        if m_hf_total:
            entry["Hf_total"] = int(m_hf_total.group(1))

        # ---- 3. PfarrHufen / spezielle Hufen ----
        details = {}

        for num, key in re.findall(r"(\d+)\s*([A-Za-zÄÖÜäöüß]+Hf[r]?)", content):
            val = int(num)
            lname = key.strip()
            details[lname] = val

        if details:
            entry.setdefault("Hf_details", {}).update(details)

        # ---- 4. Allgemeine Berufs- und Sozialgruppen ----
        # Beispiele:
        # 10 Koss
        # 1 Hirte
        # 4 B
        # 1 Schmied
        generic = re.findall(r"(\d+)\s+([A-Za-zÄÖÜäöüß][A-Za-z0-9ÄÖÜäöüß\-]*)", content)

        for num, name in generic:
            num = int(num)
            if name.endswith(("Hf", "Hfr", "PfarrHf")):
                continue
            entry[name] = num

        # ---- 5. Wahrheitswerte wie "Windmühle" ----
        singles = re.findall(r"\b([A-Za-zÄÖÜäöüß][A-Za-zÄÖÜäöüß]+mühle)\b", content)
        for s in singles:
            entry[s] = True

        # ---- 6. Wüstungen ----
        # wüste Hofstätte, wüste BStelle, wüste Hf etc.
        wust = re.findall(r"wüste\s+([A-Za-zÄÖÜäöüß]+)", content)
        if wust:
            entry["wüstungen"] = wust

        # ---- 7. Zusätzliche Angaben (Jahre innerhalb des Absatzes) ----
        additional_years = re.findall(r"(\d{3,4})", content)
        add = []
        for y in additional_years:
            if int(y) != year:
                add.append(content.strip())
                break
        if add:
            entry["additional"] = add

        results.append(entry)
        last_entry = entry

    return results


In [10]:
# Beispielaufruf

def print_economy(place_name):
    if place_name not in blocks:
        print("Ort nicht gefunden.")
        return

    data = parse_economy(blocks[place_name])

    print(f"Wirtschaftsdaten für {place_name}:\n")
    for row in data:
        print(row)


In [11]:
print_economy("AHRENSFELDE")

Wirtschaftsdaten für AHRENSFELDE:

{'year': 1375, 'Hf_total': 72, 'Koss': 10}
{'year': 1450, 'Hf_total': 72}
{'year': 1451, 'Hf_total': 72}
{'year': 1480, 'Hf_total': 72, 'Hf_details': {'PfarrHf': 4}, 'und': 38, 'Koss': 6}
{'year': 1527, 'Hf_total': 68}
{'year': 1541, 'Hf_details': {'DorfHf': 68, 'PfarrHf': 4}}
{'year': 1591, 'Hf_total': 6, 'Hf_details': {'SiebenHfr': 1, 'SechsHfr': 4, 'FünfHfr': 3, 'VierHfr': 3, 'DreiHfr': 1}, 'Setzschulze': 1, 'Koss': 6, 'Hirte': 1, 'wüste': 1, 'wüstungen': ['Hofstätte']}
{'year': 1624, 'Hf_total': 67, 'Hf_details': {'PfarrHf': 4, 'KHf': 1}, 'Koss': 6, 'Hirte': 1, 'Schmied': 1}
{'year': 1652, 'B': 4, 'Schulze': 1, 'Sohn': 1, 'Knechten': 2, 'Koss': 2}
{'year': 1664, 'Hf_total': 26, 'Koss': 1, 'Schmied': 1, 'Hirte': 1}
{'year': 1696, 'Hf_details': {'SiebenHfr': 1, 'SechsHfr': 5, 'FünfHfr': 2, 'VierHfr': 1, 'DreiHfr': 1, 'ZweiHfr': 2}, 'wüste': 2, 'wüsten': 4, 'Koss': 1, 'genutzt': 1, 'Schmied': 1, 'Hirte': 1, 'wüstungen': ['BStelle', 'Stellen']}
{'year

## 8. parse_kirche_generic



In [17]:
import re

def parse_kirche_smart(block_text):
    """
    Parser für Punkt 8 (Kirche), der automatisch nach Jahresgruppen segmentiert.
    
    Logik:
    - Punkt 8 extrahieren
    - Text glätten
    - Alle Positionen von Jahreszahlen finden
    - Für jede Jahresgruppe ein Segment erzeugen
    - Jahresgruppe = zusammenhängende Jahreszahlen, getrennt durch Kommas / Bindewörter
    """

    # -------- Punkt 8 extrahieren --------
    m = re.search(r"\b8\.\s+(.*?)(?=\n\s*\d+\.)", block_text, re.S)
    if not m:
        return {"point_8_raw": None, "segments": []}

    raw_text = m.group(1).strip()

    # Glätten (Zeilenumbrüche → Leerzeichen)
    text = re.sub(r"\s*\n\s*", " ", raw_text)

    # -------- Jahreszahlen finden --------
    # Wir suchen echte Jahre: 1000–2099
    year_pattern = r"\b(1[0-9]{3}|20[0-9]{2})\b"

    matches = list(re.finditer(year_pattern, text))
    if not matches:
        return {"point_8_raw": text, "segments": []}

    segments = []

    # -------- Segmente erzeugen --------
    for i, match in enumerate(matches):
        start = match.start()

        # Ende bestimmen:
        if i + 1 < len(matches):
            end = matches[i+1].start()
        else:
            end = len(text)

        segment_text = text[start:end].strip()

        # Jahresgruppe extrahieren
        years = [int(y) for y in re.findall(year_pattern, segment_text)]

        segments.append({
            "raw": segment_text,
            "years": years
        })

    return {
        "point_8_raw": text,
        "segments": segments
    }


In [19]:
def print_kirche(place_name):
    if place_name not in blocks:
        print("Ort nicht gefunden.")
        return

    data = parse_kirche_smart(blocks[place_name])
    print(data)


In [21]:
print_kirche("BERNAU")

{'point_8_raw': "Bt Brandenburg, 1296 Propstei; 1375, 1545, 1'::-00, 1900 !!.K (MarienK), Sedes bzw. Insp bzw. Sup B., mit TK Schönow, Schönwalde (bis 1867), vorübergehend auch Bärnicke (1716 bis 1719), seit 1930 mit TK Ladeburg; Patr: seit 1545 der Rat der Stadt; seit 1699 französisch-reformierte Gern, seit 1751 deutsch-reformierte, beide 1210 vereinigt und 1.::-25 mit der reformierten Gem Französisch-Buchholz, seit 1910 Vereinigung der deutsch-evangelischen KGem in Französisch- Buchholz mit den französisch-reformierten Gem daselbst und in B. unter einem Pfarramt; Patr: Kg.", 'segments': [{'raw': '1296 Propstei;', 'years': [1296]}, {'raw': '1375,', 'years': [1375]}, {'raw': "1545, 1'::-00,", 'years': [1545]}, {'raw': '1900 !!.K (MarienK), Sedes bzw. Insp bzw. Sup B., mit TK Schönow, Schönwalde (bis', 'years': [1900]}, {'raw': '1867), vorübergehend auch Bärnicke (', 'years': [1867]}, {'raw': '1716 bis', 'years': [1716]}, {'raw': '1719), seit', 'years': [1719]}, {'raw': '1930 mit TK Lad

## 9. place_monuments

In [22]:
# Baudenkmale-Parser

def parse_baudenkmale(block_text):
    """
    Parser für Punkt 9 (Baudenkmale) aus HOL.
    - extrahiert Punkt 9
    - erkennt mehrere Denkmale
    - trennt Titel + Beschreibung generisch
    """

    # -------- Punkt 9 extrahieren --------
    m = re.search(r"\b9\.\s+(.*?)(?=\n\s*\d+\.)", block_text, re.S)
    if not m:
        return []

    text = m.group(1).strip()

    # Zeilen glätten
    text = re.sub(r"\s*\n\s*", " ", text)

    # Eventuelle Endpunkte vereinheitlichen
    text = text.replace("–", "-").strip()

    # -------- Denkmale segmentieren --------
    # Annahme: getrennt durch ; oder mehrere Punkte
    raw_segments = re.split(r";\s*|\.\s+(?=[A-ZÄÖÜ])", text)

    monuments = []

    for seg in raw_segments:
        seg = seg.strip().rstrip(".")
        if not seg:
            continue

        # Titel = alles BIS zum ersten Komma
        m2 = re.match(r"([^,]+),\s*(.*)", seg)
        if m2:
            title = m2.group(1).strip()
            desc = m2.group(2).strip()
        else:
            # Falls kein Komma → alles Titel
            title = seg
            desc = ""

        monuments.append({
            "title": title,
            "description": desc,
            "notes": None  # für spätere Quellen, falls gebraucht
        })

    return monuments


In [42]:
def print_monuments(place_name):
    if place_name not in blocks:
        print(f"Ort '{place_name}' nicht gefunden!")
        return

    monuments = parse_baudenkmale(blocks[place_name])

    if not monuments:
        print(f"Keine Baudenkmale (Punkt 9) für {place_name} gefunden.")
        return

    print(f"Baudenkmale für {place_name}:\n")
    for i, m in enumerate(monuments, start=1):
        print(f"{i}. {m['title']}")
        if m['description']:
            print(f"   Beschreibung: {m['description']}")
        if m['notes']:
            print(f"   Hinweise: {m['notes']}")
        print()


In [43]:
# Beispielaufruf

print_monuments("AHRENSFELDE")

Baudenkmale für AHRENSFELDE:

1. Ma FeldsteinK
   Beschreibung: rechteckiger Saalbau mit w eingezogenem Turm, 1768 barock umgebaut, 1875/76 weitere Bauveränderun- gen



## 10. bevoelkerung

In [44]:
import re

def clean_number(s):
    """Repariert typische OCR-Fehler in Jahres- und Einwohnerzahlen."""
    replacements = {
        'G': '6', 'E': '6', 'L': '1', 'l': '1', 'O': '0'
    }
    for bad, good in replacements.items():
        s = s.replace(bad, good)
    return re.sub(r"[^\d]", "", s)


def parse_bevoelkerung_advanced(block_text):
    """
    Sehr robuster Parser für Punkt 10: Bevölkerung.
    Erkennt:
    - 1515: gegen 4000
    - 1640-50: 500-700
    - 1G35: 4E98
    - 1925: 993t: und 370 (Bernau Süd) und 104 ...
    """

    # Punkt 10 extrahieren (auch wenn letzter Punkt!)
    m = re.search(r"\b10\.\s+(.*?)(?=\n\s*\d+\.|$)", block_text, re.S)
    if not m:
        return []

    text = m.group(1)

    # Suche alle Jahr: danach-Text - Paare
    # Beispiel matcht alles bis zum nächsten Jahr
    raw_entries = re.findall(r"(\d{3,4}(?:[-/]\d{2,4})?)\s*:\s*([^:,]+)", text)

    results = []

    for raw_year, raw_value in raw_entries:

        # -------- Jahr bereinigen ----------
        year_str = clean_number(raw_year)

        # Bereichsjahre wie 1640-50 → wir nehmen erstes Jahr
        if "-" in year_str:
            year_str = year_str.split("-")[0]

        try:
            year = int(year_str)
        except:
            continue  # komplett unbrauchbar → skip

        # -------- Einwohnerzahl bereinigen ----------
        # Finde erste Zahl im Wertebereich
        nums = re.findall(r"\d[\d\s]*", raw_value)
        if not nums:
            continue

        inhabitants = int(clean_number(nums[0]))

        results.append({
            "year": year,
            "inhabitants": inhabitants,
            "raw": raw_value.strip()
        })

    return results


In [39]:
def print_population(place_name):
    if place_name not in blocks:
        print(f"Ort '{place_name}' nicht gefunden!")
        return

    data = parse_bevoelkerung_advanced(blocks[place_name])

    if not data:
        print(f"Keine Bevölkerungsdaten (Punkt 10) für {place_name} gefunden.")
        return

    print(f"Bevölkerungszahlen für {place_name}:\n")
    for e in data:
        print(f"{e['year']}: {e['inhabitants']}  (raw: {e['raw']})")


In [45]:
print_population("AHRENSFELDE")

Bevölkerungszahlen für AHRENSFELDE:

1734: 125  (raw: 125)
1772: 112  (raw: 112)
1801: 174  (raw: 174)
1817: 154  (raw: 154)
1840: 210  (raw: 210)
1858: 330  (raw: 330)
1895: 514  (raw: 514)
1925: 805  (raw: 805)
1939: 1887  (raw: 1887)
1946: 1795  (raw: 1795)
1964: 1826  (raw: 1826)
1971: 1730  (raw: 1730.)


In [49]:
print_population("BERNAU")

Bevölkerungszahlen für BERNAU:

1515: 4000  (raw: gegen 4000)
1625: 2500  (raw: um 2500)
164050: 500  (raw: 500-700)
1733: 974  (raw: 974)
1750: 1438  (raw: 1438)
1772: 1564  (raw: 1564)
1790: 1639  (raw: 1639)
1500: 1791  (raw: 1791)
1317: 1  (raw: 1G35 mit Kleine Mühle)
1040: 2672  (raw: 2672)
1925: 993  (raw: 993t)
1939: 13  (raw: 13 ~53)
1946: 12984  (raw: 12 984)
1964: 13909  (raw: 13 909)
1971: 15463  (raw: 15 463.)
